# Fine-tuning Generative AI Model for Dialogue Summarization

This is a practice notebook by Mark Redito. This is a copy of the same notebook used in the course, [Generative AI with LLMs](https://www.deeplearning.ai/courses/generative-ai-with-llms/) with Redito's comments. All rights are reserved by the authors.

# 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

In this notebook, you will fine-tune an existing LLM from Hugging Face for enhanced dialogue summarization. You will use the [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model, which provides a high quality instruction tuned model and can summarize text out of the box. To improve the inferences, you will explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.


**1.1 - Set up Kernel and Required Dependencies**

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \ #used for eval
    loralib==0.1.1 \ #used for parameter efficient training
    peft==0.3.0 --quiet #used for parameter efficient training

Importing other necessary components:

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

`AutoModelForSeq2SeqLM` is our model. `AutoTokenizer` is our tokenizer. `TrainingArguments` and `Trainer` is used to simplify the training of the model.

**1.2 - Load Dataset and LLM**
We are going to experiment with the `DialogSum` Hugging Face dataset. Which containts 10k+ dialogues with manual labeled summaries and topics.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Above also prints out Dataset features and stats such as: id, dialogue, summary, topic, and num_rows

Next, we load the pre-trained FLAN-T5 model and its tokenizer directly from HuggingFace. This will be a small version of the model. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by the model.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Next, lets print out the model parameters and look at how many of those parameters are trainable.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

Notice that the percentage of trainable model parameters are 100%, meaning all of the model weights are trainable.

**1.3 - Test the Model with Zero Shot Inferencing**

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

#below is a prompt template
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

The results show that model is decent at summarization and could be better with fine tuning.

# 2 - Perform Full Fine Tuning

**2.1 - Preprocess the Dialog-Summary Dataset**
In this stage we need to convert the dialog-summary (prompt and response) pairs into explicit instructions for the model. We prepend an instruction to the start of the dialog with `Summarize the following conversation` and to the start of the summary with `Summary:` as follows:



```
Summarize the following conversation.
  Chris: This is his part of the conversation.
  Antje: This is her part of the conversation.

Summary:
```

Training response(Summary):

`Both Chris and Antje participated in the conversation.`

And then we pre-process the prompt-response dataset into tokens and pull out their `input_ids` (1 per token)


In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

To save some time, we will subsample the dataset:

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Lets check the shapes of all three parts of the dataset:

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Above is a print out of the dataset. Here we have split the dataset into training (used for training), Validation (to validate the model) and finally Test (which we're holding out for testing). Looks like we're ready for some fine-tuning!

**2.2 Fine-Tune the Model with the Preprocessed Dataset**

We utilize the built-in Hugging Face `Trainer` ([Docs here](https://huggingface.co/docs/transformers/main_classes/trainer)).

Notice that we set the `max_steps` and `num_train_epochs` to 1. This saves us time and compute costs for the purpose of this practice notebook.

Cool. lets start the training process:

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

Lets go!!!

In [ ]:
trainer.train()

Training a fully fine-tuned model would take a few hours on a GPU. To save time, we download a checkpoint of the fully fine-tuned model to use in the rest of this notebook. This fully fine-tuned model with also be referred to as the **Instruct Model** in this lab.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

The size of the model is ~1Gb

In [ ]:
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin

Here we create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:
Notice that we use `bfloat16` for compute efficiency.

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

**2.3 - Evaluate the Model Qualitatively (Human Evaluation)**

"Is my model behaving the way it is supposed to?" is usually a good starting point.

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

Running the prompts above gives a sense of the model's output in relation to Baseline Human Summary and the Original Model. It seems like the Instruct Model (The model that was finetuned), has better responses.

**2.4 - Evaluate the Model Quantitavely (with ROUGE Metric)**

The [Rogue metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations procued by models. It compares summarazations to a baseline summary which is usually created by a human. While imperfect, it seems to indicate the overall increate in summarization effectiveness (capturing nuances and details) that we achieved through fine-tuning.

In [ ]:
rouge = evaluate.load('rouge')

Lets generate the outputs for the sample of the test dataset. In this case 10 dialogues and summaries to save time. Then we save the results.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

the output above shows a table that compares `human_baseline_summaries` with `original_model_summaries` and `instruct_model_summaries`.

Next we evaluate the model through ROGUE metrics. Across the ROGUE metrics, there seems to be a great improvement.

In [ ]:
results = pd.read_csv("data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

Again, it seems like our model greatly improved its ROGUE scores.

In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

The output of the code above shows the absolute percentage improvement of Instruct Model over Original Model. Looks pretty good!

# 3 - Perform Parameter Efficient Fine-Tuning (PEFT)

Now, lets perform PEFT. In comparison to the approach above which was "full fine-tuning", PEFT is much more efficient (low compute resources) and yields comparable results as full fine tuning.


Additional notes from the og notebook:


> PEFT is a generic term that includes **Low-Rank Adaptation (LoRA)** and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

> That said, at inference time, the LoRA adapter needs to be reunited and combined with its original LLM to serve the inference request.  The benefit, however, is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.


**3.1 - Setup the PEFT/LoRA model for Fine-Tuning**

Lets setup the PEFT/LoRA. We are going to "freeze" the underlying LLM and only train the adapter. Notice the rank `r=32` hyper-parameter. This defines the rank/dimension of the adapter to be trained. *Typically this is a lower number but we'll see what results we get from this.*

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Then we add the LoRA adapter layers/parameters to the original LLM to be trained.

In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

As you notice from the output above. The percentage of the trainable model parameters is 1.41%. Compare that to the trainable parameters we did previously (above scroll up) for full fine-tuning, which was 100%. This saves us a lot of time and compute resources.

**3.2 - Train PEFT Adapter**

Here, lets define some arguments and create a `trainer` instance.

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Ok, were good to go. Lets hit train!

In [ ]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

The training performed above was perfomed only on a subset of data. Tull load a fully trained PEFT model, we need to load a checkpoint of a PEFT model from S3

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

Now lets look at the size of this model compared to the original model:

In [ ]:
!ls -al ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

Now lets prepare this model by adding an adapter to the original FLAN-T5 model. We are setting `is_trainable=False` because the plan is only to perform inference with this PEFT model. In another case of preparing the model for further training, we wouldld set this as `is_trainable=True`.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

The number of trainable parameters is 0 because we set `is_trainable=False`.

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

**3.3 - Evaluate the Model Qualitatively (Human Evaluation)**

Ok, lets test this model out:

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

The model output is not bad. There's a little more detail to the summary.

**3.4 - Evaluate the Model Quantitatively (with ROGUE Metric)**

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

The output above shows a table comparing `human_baseline_summaries`, `original_model_summaries`, `instruct_model_summaries` and `peft_model_summaries`. I noticed that the outputs of the `peft_model_summaries` and `instruct_model_summaries` are quite similar to each other.

Ok, now lets compute the ROGUE score for this subset of data.

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

Here we find that the PEFT model score and INSTRUCT model score are close to each other. Pretty impressive performance boost from PEFT while not requiring the same compute resources as INSTRUCT. The training process was also much more easier!

In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

The results show not much improvement from full fine-tuning but the benefits of PEFT outweigh the slightly lower performance metrics.

Now lets calculate the improvement of PEFT over the original model:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Pretty good improvement!

Now lets calculate the improvement of PEFT over a full fine tuned model:

In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Percentage improvement of PEFT MODEL over the INSTRUCT MODEL, shows a slight decrease. But considering that PEFT requires less computing and memory resources is a good tradeoff.

This shows how PEFT can be a better approach for practitioners who are constrained with compute and memory resources. I'll definitely keep exploring using PEFT in future LLM projects!